In [1]:
import os
os.environ["LANGGRAPH_DISABLE_DAG_RENDER"] = "1"
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from google import genai

c:\Users\ANUP N\OneDrive\Desktop\LangGraph Tut\langgraph\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [2]:
load_dotenv()

True

In [6]:
api_key = os.getenv("GEMINI_API_KEY_SELF") # or os.getenv("GEMINI_API_KEY")
if not api_key:
	raise EnvironmentError("GEMINI_API_KEY2 (or GEMINI_API_KEY) not set in environment. Add it to your .env or export it.")

# client = genai.Client(api_key=api_key)

In [7]:
model = ChatOpenAI(
   model="gemini-2.5-flash",
   api_key=api_key,
   base_url="https://generativelanguage.googleapis.com/v1beta/openai"
)

In [3]:
class Blogstate(TypedDict):
    title: str
    outline: str
    content: str

In [8]:
def create_outline(state: Blogstate)-> Blogstate:
    title= state['title']
    prompt= f'Generate the detailed outline for a blog on the topic- {title}'
    state['outline']= model.invoke(prompt)
    return state

In [10]:
def create_blog(state: Blogstate) -> Blogstate:
    title= state['title']
    outline= state['outline']
    prompt_blog= f'Write 500 words blog on the title {title} using following outline:\n {outline}'
    content= model.invoke(prompt_blog)
    state['content']= content
    return state

In [16]:
graph= StateGraph(Blogstate)
# nodes
graph.add_node('outline_gen', create_outline )
graph.add_node('generate_blog',create_blog)


#edges
graph.add_edge(START, 'outline_gen')
graph.add_edge('outline_gen', 'generate_blog')
graph.add_edge('generate_blog', END)

workflow= graph.compile()

In [17]:
initial_state= {'title': 'Top 10 movies in 2025 year'}
final_state= workflow.invoke(initial_state)

In [18]:
print(final_state)

{'title': 'Top 10 movies in 2025 year', 'outline': AIMessage(content='Okay, here\'s a detailed outline for a blog post on "Top 10 Movies in 2025," designed to be engaging, comprehensive, and forward-looking, given that 2025 is in the future.\n\n---\n\n## Blog Title Options:\n\n*   **Option 1 (Catchy & Anticipatory):** Unveiling the Future: Our Top 10 Most Anticipated Movies of 2025\n*   **Option 2 (Direct & Exciting):** The Next Big Screen Revolution: Top 10 Must-See Movies Expected in 2025\n*   **Option 3 (Intriguing):** Beyond the Horizon: Predicting the Top 10 Cinematic Masterpieces of 2025\n\n---\n\n## **Detailed Blog Post Outline: Unveiling the Future: Our Top 10 Most Anticipated Movies of 2025**\n\n---\n\n### **I. Introduction (Approx. 150-200 words)**\n\n*   **A. Engaging Hook:**\n    *   Start with the excitement of cinema and the thrill of looking ahead. "As the credits roll on one year, the anticipation for the next immediately begins to build in the world of film."\n    *   

In [20]:
final_state['outline'].content

'Okay, here\'s a detailed outline for a blog post on "Top 10 Movies in 2025," designed to be engaging, comprehensive, and forward-looking, given that 2025 is in the future.\n\n---\n\n## Blog Title Options:\n\n*   **Option 1 (Catchy & Anticipatory):** Unveiling the Future: Our Top 10 Most Anticipated Movies of 2025\n*   **Option 2 (Direct & Exciting):** The Next Big Screen Revolution: Top 10 Must-See Movies Expected in 2025\n*   **Option 3 (Intriguing):** Beyond the Horizon: Predicting the Top 10 Cinematic Masterpieces of 2025\n\n---\n\n## **Detailed Blog Post Outline: Unveiling the Future: Our Top 10 Most Anticipated Movies of 2025**\n\n---\n\n### **I. Introduction (Approx. 150-200 words)**\n\n*   **A. Engaging Hook:**\n    *   Start with the excitement of cinema and the thrill of looking ahead. "As the credits roll on one year, the anticipation for the next immediately begins to build in the world of film."\n    *   Mention the constant evolution of storytelling and technology.\n*   *

In [24]:
print(final_state['content'].content)

## Unveiling the Future: Our Top 10 Most Anticipated Movies of 2025

As the credits roll on one year, the anticipation for the next immediately begins to build in the vibrant world of film. With storytelling constantly evolving and cinematic technology pushing new boundaries, cinephiles are always eager to discover what blockbusters, dramas, and animated marvels await them.

While crystal balls are still in development, we've scoured the rumor mills, official announcements, and industry trends to bring you our speculative, yet highly confident, predictions for the movies that will define 2025. This list offers a sneak peek at the diverse slate expected, from epic sci-fi sagas and superhero tentpoles to intimate dramas and groundbreaking animation. Get ready to mark your calendars (tentatively!) as we unveil the films we can't wait to see.

---

### The Top 10 Most Anticipated Movies of 2025

1.  **The Galactic Chronicles: Part III**
    *   **Genre:** Sci-Fi Epic
    *   **Buzz Factor: